# Prototyping Decodoku 2022

This notebook contains work in progress for a sequel to the citizen science game [Decodoku](https://decodoku.blogspot.com/), made as a resource for lectures in the [QEC Summer School 2022](https://wwwpoc.ibm.com/quantum-computing/summer-school/).

We'll use the [Jupylet](https://jupylet.readthedocs.io/en/latest/) port of Qisge (which is just a file in this repo).

In [1]:
import qisge

And we'll need some randomness.

In [2]:
import random

The only image needed is the background grid. 

In [3]:
images = qisge.ImageList([
    'grid8.png'
    ])

background = qisge.Sprite(0,size=16.2,x=6,y=0)

The syndrome will be displayed with text boxes. Note the weird magic numbers used in the `x` and `y` coordinates. These are due to the WIP nature of the Jupylet port of Qisge, and will be fixed in future.

In [4]:
syndrome = {}
for x in range(8):
    for y in range(8):
        syndrome[x,y] = qisge.Text('·',0,1,x=1.95*x+7,y=1.95*y+.9)

Errors are associated with pairs of neighbouring squares on the grid, so we'll make a list of them all.

In [5]:
pairs = []
for x in range(8):
    for y in range(8):
        if x+1 <8:
            pairs.append([(x,y),(x+1,y)])
        if y+1<8:
            pairs.append([(x,y),(x,y+1)])

Now for the `next_frame` function. When any arrow key is pressed, each possible error will occur in each frame with probability `p`. When no arrow key is pressed, nothing happens.

In [6]:
p = 0.001

def next_frame(userinput):
    
    global syndrome, pairs
    
    # action happens if any arrow key is pressed
    if {0,1,2,3}.intersection(set(userinput['key_presses'])):
        for pair in pairs:
            if random.random()<p:
                pair = random.choice(pairs)
                for s in pair:
                    if syndrome[s].text=='1':
                        syndrome[s].text = '-'
                    else:
                        syndrome[s].text = '1'   

Now we run it to see it in action.

In [7]:
@qisge._app.run_me_every(1/qisge.FPS)
def game_loop(ct, dt):
    next_frame(qisge.update())
    
qisge._app.run()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

This will show all syndromes initially as `·`. When they detect an error, this will change to `1`. If a syndrome does not detect an error after it previously has done, this will be denoted with `-`.

So just watch some errors happen. That's all for now!